# Classification: Vinho Verde Wine Quality

The following project is based on the Wine Quality dataset found on UCI's Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/wine+quality). It contains two data sets (white and red wine), with 12 attributes each. 

The main task is to create a model that would predict the quality of the wine, which is the score between 3 and 9. I would assume the original scale is between 0 and 10, but our dataset does not contain scores 0, 1, 2 and 10. 

The main difference between Titanic project and this one is that in the former there are only 2 possible cases to predict: survived or not. Since classifying wines into 7 possible outcomes is an extremely difficult task, we will try to find the number of classes that will give us a lot of information, yet will have high performance and accuracy.

Note: since there are two dataset, we are going to combine them in one but add an extra column reflecting wine type.

## Plan of Attack

<ol>
    <li>Data Cleaning: check for</li>
        <ul>
            <li>Null values</li>
            <li>Missing values</li>
            <li>Duplicates</li>
        </ul>
    <li>Exploratory Analysis: helps us to understand the dataset better</li>
    <li>Classification model</li>
        <ul>
            <li>7 classes</li>
            <li>4 classes</li>
            <li>2 classes</li>
        </ul>
    <li>Final words</li>
</ol>

## Importing Libraries

In [1]:
# data manipulation
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np

# data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

## Loading Datasets

Firstly, we need to read csv files. While csv stands for comma-separated values, this particular documents have values separated by ';' so we have to include 'sep' parameter in read_csv() method.

In [2]:
red_wine = pd.read_csv('datasets/winequality-red.csv', sep=';')
white_wine = pd.read_csv('datasets/winequality-white.csv', sep=';')

Since we want to unite both dataframes in one, we need to create a custom function which combines both dataframes and creates a new column type that records from which dataframe whine comes from (red vs white).

In [3]:
def unite_wines(red_wine, white_wine):
    red_wine['is white'] = 0
    white_wine['is white'] = 1
    return pd.concat([red_wine, white_wine], sort=True).reset_index(drop=True)

In [4]:
df_wine = unite_wines(red_wine, white_wine)
df_wine.head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,is white,pH,quality,residual sugar,sulphates,total sulfur dioxide,volatile acidity
0,9.400,0.076,0.000,0.998,7.400,11.000,0,3.510,5,1.900,0.560,34.000,0.700
1,9.800,0.098,0.000,0.997,7.800,25.000,0,3.200,5,2.600,0.680,67.000,0.880
2,9.800,0.092,0.040,0.997,7.800,15.000,0,3.260,5,2.300,0.650,54.000,0.760
3,9.800,0.075,0.560,0.998,11.200,17.000,0,3.160,6,1.900,0.580,60.000,0.280
4,9.400,0.076,0.000,0.998,7.400,11.000,0,3.510,5,1.900,0.560,34.000,0.700


## Data Cleaning

In [5]:
df_wine['quality'].unique()

array([5, 6, 7, 4, 8, 3, 9], dtype=int64)

In [6]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   alcohol               6497 non-null   float64
 1   chlorides             6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   density               6497 non-null   float64
 4   fixed acidity         6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   is white              6497 non-null   int64  
 7   pH                    6497 non-null   float64
 8   quality               6497 non-null   int64  
 9   residual sugar        6497 non-null   float64
 10  sulphates             6497 non-null   float64
 11  total sulfur dioxide  6497 non-null   float64
 12  volatile acidity      6497 non-null   float64
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


We can see that there are 6497 entries in the dataframe, and each column has 6497 non-null values. So this means that there are no missing values and no null values in the dataframe. 

In [7]:
df_wine = df_wine.drop_duplicates()
print(len(df_wine))

5320


We have started with 6497 entries and after dropping duplicates we have reduced the number of entries to 5320, which is 18% decrease in size. That's a lot of duplicates!

Now we need to reset indices since after dropping duplicates we have removed some indices. If we don't reset them, looping through the dataframe would become problematic.

In [8]:
df_wine = df_wine.reset_index(drop=True)


## Exploratory Analysis

First we can use describe() function to have a look at the data.

In [9]:
df_wine.describe()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,is white,pH,quality,residual sugar,sulphates,total sulfur dioxide,volatile acidity
count,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000,5320.000
mean,10.549,0.057,0.318,0.995,7.215,30.037,0.745,3.225,5.796,5.048,0.533,114.109,0.344
std,1.186,0.037,0.147,0.003,1.320,17.805,0.436,0.160,0.880,4.500,0.150,56.774,0.168
min,8.000,0.009,0.000,0.987,3.800,1.000,0.000,2.720,3.000,0.600,0.220,6.000,0.080
25%,9.500,0.038,0.240,0.992,6.400,16.000,0.000,3.110,5.000,1.800,0.430,74.000,0.230
50%,10.400,0.047,0.310,0.995,7.000,28.000,1.000,3.210,6.000,2.700,0.510,116.000,0.300
75%,11.400,0.066,0.400,0.997,7.700,41.000,1.000,3.330,6.000,7.500,0.600,153.250,0.410
max,14.900,0.611,1.660,1.039,15.900,289.000,1.000,4.010,9.000,65.800,2.000,440.000,1.580


Lets see how the data changes with respect to the quality of the wine. 

In [10]:
df_wine.groupby('quality').describe()

alcohol                                                 chlorides  \
           count   mean   std    min    25%    50%    75%    max     count   
quality                                                                      
3         30.000 10.215 1.106  8.000  9.625 10.150 11.000 12.600    30.000   
4        206.000 10.215 0.991  8.400  9.400 10.100 10.900 13.500   206.000   
5       1752.000  9.872 0.828  8.000  9.300  9.600 10.300 14.900  1752.000   
6       2323.000 10.649 1.112  8.400  9.800 10.500 11.400 14.000  2323.000   
7        856.000 11.511 1.116  8.600 10.800 11.500 12.325 14.200   856.000   
8        148.000 11.912 1.078  8.500 11.200 12.200 12.700 14.000   148.000   
9          5.000 12.180 1.013 10.400 12.400 12.500 12.700 12.900     5.000   

               ... total sulfur dioxide         volatile acidity              \
         mean  ...                  75%     max            count  mean   std   
quality        ...                                                             
3       0.077  ...              193.250 440.000           30.000 0.517 0.342   
4       0.061  ...              150.250 272.000          206.000 0.462 0.232   
5       0.066  ...              167.000 344.000         1752.000 0.394 0.183   
6       0.054  ...              154.000 294.000         2323.000 0.316 0.149   
7       0.045  ...              134.000 289.000          856.000 0.292 0.117   
8       0.040  ...              135.500 212.500          148.000 0.303 0.118   
9       0.027  ...              124.000 139.000            5.000 0.298 0.058   

                                       
          min   25%   50%   75%   max  
quality                                
3       0.170 0.253 0.415 0.633 1.580  
4       0.110 0.280 0.385 0.610 1.130  
5       0.100 0.260 0.340 0.500 1.330  
6       0.080 0.220 0.270 0.370 1.040  
7       0.080 0.210 0.280 0.350 0.915  
8       0.120 0.220 0.280 0.360 0.850  
9       0.240 0.260 0.270 0.360 0.360  

[7 rows x 96 columns]

We can see how imbalanced is the dataset. There are much observations of average wine compare to very poor and high quality wine. This is one of the things we would have to address later on.

Now lets create dummy variables for the dependent variable.

In [11]:
dummys = pd.get_dummies(df_wine['quality'])
dummys.head()

,3,4,5,6,7,8,9
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0
4,0,0,1,0,0,0,0


In [16]:
new_df = pd.concat([df_wine, dummys], axis=1)
new_df.head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,is white,pH,quality,residual sugar,sulphates,total sulfur dioxide,volatile acidity,3,4,5,6,7,8,9
0,9.400,0.076,0.000,0.998,7.400,11.000,0,3.510,5,1.900,0.560,34.000,0.700,0,0,1,0,0,0,0
1,9.800,0.098,0.000,0.997,7.800,25.000,0,3.200,5,2.600,0.680,67.000,0.880,0,0,1,0,0,0,0
2,9.800,0.092,0.040,0.997,7.800,15.000,0,3.260,5,2.300,0.650,54.000,0.760,0,0,1,0,0,0,0
3,9.800,0.075,0.560,0.998,11.200,17.000,0,3.160,6,1.900,0.580,60.000,0.280,0,0,0,1,0,0,0
4,9.400,0.075,0.000,0.998,7.400,13.000,0,3.510,5,1.800,0.560,40.000,0.660,0,0,1,0,0,0,0


In [17]:
new_df.drop(['quality'], axis=1, inplace=True)
new_df.head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,is white,pH,residual sugar,sulphates,total sulfur dioxide,volatile acidity,3,4,5,6,7,8,9
0,9.400,0.076,0.000,0.998,7.400,11.000,0,3.510,1.900,0.560,34.000,0.700,0,0,1,0,0,0,0
1,9.800,0.098,0.000,0.997,7.800,25.000,0,3.200,2.600,0.680,67.000,0.880,0,0,1,0,0,0,0
2,9.800,0.092,0.040,0.997,7.800,15.000,0,3.260,2.300,0.650,54.000,0.760,0,0,1,0,0,0,0
3,9.800,0.075,0.560,0.998,11.200,17.000,0,3.160,1.900,0.580,60.000,0.280,0,0,0,1,0,0,0
4,9.400,0.075,0.000,0.998,7.400,13.000,0,3.510,1.800,0.560,40.000,0.660,0,0,1,0,0,0,0


In [18]:
from sklearn.preprocessing import MinMaxScaler
x = new_df.loc[:, new_df.columns].values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_normalized = pd.DataFrame(data=x_scaled, columns=new_df.columns)

In [19]:
df_normalized.head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,is white,pH,residual sugar,sulphates,total sulfur dioxide,volatile acidity,3,4,5,6,7,8,9
0,0.203,0.111,0.000,0.206,0.298,0.035,0.000,0.612,0.020,0.191,0.065,0.413,0.000,0.000,1.000,0.000,0.000,0.000,0.000
1,0.261,0.148,0.000,0.187,0.331,0.083,0.000,0.372,0.031,0.258,0.141,0.533,0.000,0.000,1.000,0.000,0.000,0.000,0.000
2,0.261,0.138,0.024,0.191,0.331,0.049,0.000,0.419,0.026,0.242,0.111,0.453,0.000,0.000,1.000,0.000,0.000,0.000,0.000
3,0.261,0.110,0.337,0.210,0.612,0.056,0.000,0.341,0.020,0.202,0.124,0.133,0.000,0.000,0.000,1.000,0.000,0.000,0.000
4,0.203,0.110,0.000,0.206,0.298,0.042,0.000,0.612,0.018,0.191,0.078,0.387,0.000,0.000,1.000,0.000,0.000,0.000,0.000


Regression

In [12]:
X = df_wine.drop(['quality'], axis=1)
y = df_wine['quality']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [22]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[  0   1   4   1   0   0   0]
 [  0   5  31  18   0   0   0]
 [  0   5 283 140   3   0   0]
 [  0   1 127 399  35   0   0]
 [  0   0  10 120  69   3   0]
 [  0   0   0  27   8   0   0]
 [  0   0   0   1   0   0   0]]


0.5855925639039504